# Цели и поставленные задачи

Нужно выявить определяющие успешность игры закономерности. Это позволит сделать ставку на потенциально популярный продукт и спланировать рекламные кампании. Для этого необходимо обработать данные в нашей базе данных, выделить актуальные данные, с которыми мы будем работать, определить предпочтения рынков Северной Америки, Японии и остального мира. Какие платформы более интересны в каком рынке, какие жанры предпочтительней. Понять есть ли зависимость продаж от рейтингов игроков или критиков. Провести анализ продаж для каждого региона и понять предпочтения рынков. Также нам необходимо проверить две гипотезы: 1. Средние пользовательские рейтинги платформ Xbox One и PC одинаковые; 2. Средние пользовательские рейтинги жанров Action и Sports разные.

# Загрузка датасета и библиотек

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
from scipy import stats as st
import matplotlib.pyplot as plt
from IPython.display import display
from scipy.stats import levene

In [ ]:
try:
    games = pd.read_csv('/datasets/games.csv')
except:
    games = pd.read_csv('games.csv')
games.info()
games

Довольно большое количество пропусков. Названия столбцов с заглавными и строчными буквами, что неудобно для работы с ними. Рейтинги в разных типах (два из трёх в object), что необходимо исправить.

# Подготовка данных

## Название столбцов

In [ ]:
#приведём все названия столбцов к нижнему регистру
games.columns = games.columns.str.lower()
games.info()

## Типы данных

In [ ]:
games.loc[games['year_of_release'].isnull() == True]

In [ ]:
#не удалось привести значения к int в связи с пропусками в значениях, которые мы не можем корректно заполнить
#заменим тип на Int64, которые позволяет пропуски
games['year_of_release'] = games['year_of_release'].astype('Int64')

#games['user_score'] = games['user_score'].astype('float')
len(games.query('user_score == "tbd"')) / len(games)

In [ ]:
#в оценках пользователей есть значения 'tbd', говорящие, что рейтинг ещё не проставлен. заменим эти значения на пропуски.
games['user_score'] = games['user_score'].replace('tbd', 'nan')
games['user_score'] = games['user_score'].astype('float')

In [ ]:
#оценки критиков поделим на 10, чтобы привести их к значениям оценок пользователей
games['critic_score'] = games['critic_score'] / 10

## Поиск дубликатов и пропусков

In [ ]:
games.duplicated().sum()

In [ ]:
games.isnull().sum()

In [ ]:
#две игры без названия
games.loc[games['name'].isnull() == True]

In [ ]:
#довольно старые, без рейтинга. сбросим их.
games.dropna(subset=['name'], inplace=True)
games.isnull().sum()

In [ ]:
#для анализа будет необходим год выхода игры, а затем мы будем делать по актуальности данных. 
#так как корректно заполнить год нет возможности, то 269 из 16713 записей можно выкинуть.
games.dropna(subset=['year_of_release'], inplace=True)

In [ ]:
games.isnull().sum()

Пропуски рейтнигов нет возможности заполнить данными, которые потом нам не исказят картину корелляции между рейтингами и продажами.

## Создание столбца с суммарными продажами

In [ ]:
games['total_sales'] = games['na_sales'] + games['eu_sales'] + games['jp_sales'] + games['other_sales']
games

# Анализ данных

## Выход игр по годам

In [ ]:
#построим гистограмму распределения выхода игр по годам
fig = px.histogram(games, x = 'year_of_release', title='Распределение выхода игр по годам', marginal = 'box')
fig.show()

Как мы видим, игровая индустрия пошла в рост с 1991 года и пиком по количеству релизов игр был 2008 год.

## Суммарные продажи по годам и платформам

In [ ]:
#выведем две сводные таблицы: суммарных продаж за каждый год и суммарных продаж по каждой платформе
display(games
       .pivot_table(index = 'year_of_release', values = 'total_sales', aggfunc = 'sum')
       .sort_values(by = 'total_sales', ascending = False)
        .head(10)
)

display(games
        .pivot_table(index = 'platform', values = 'total_sales', aggfunc='sum')
        .sort_values(by = 'total_sales', ascending = False)
        .head(10)
)

Как мы видим, лидером по продажам явлется платформа PlayStation 2. 2 и 3 место занимают XBox 360 и её прямой конкурент PlayStation 3 с небольшой разницой.

In [ ]:
#создадим функцию для вызова heatmap
def pivot_heatmap(data, col, val):
    fig, ax = plt.subplots(figsize=(10,10))
    sns.heatmap(data.pivot_table(index = 'year_of_release', columns = col, values = val, aggfunc='sum'),
        square = True, linewidths=.5, ax=ax
    )
    fig.show()
    
#посмотрим на общую картину продаж на платформах в разные годы    
pivot_heatmap(games, 'platform', 'total_sales')

Как можно увидеть из карты немногие платформы держатся на рынке более 10 лет (за исключение ПК, конечно). При этом новое поколение пересекается с предыдущим на протяжении 4-5 лет. Пиковые значения продаж приходят на платформу как раз на конец существование предыдущего поколения, то есть на 4-5 год.

In [ ]:
(games.loc[:,['na_sales', 'jp_sales', 'eu_sales', 'other_sales']]
    .sum()
    .plot.pie(
    autopct="%.2f",
    fontsize=20,
    figsize=(10, 10),
    subplots=True,
    title='Доли продаж по рынкам в мире')
)

Североамериканский рынок самый крупный по продажам из выборки и составляет почти половину от продаж во всё мире.

In [ ]:
#нас интересуют игры и платформы за последние 4 года, чтобы захватить пиковые продажи на предыдущем поколении платформ
#эти платформы вероятно на следующий год или через один уже перестанут выпускать релизы, но дадут нам акутальную информацию
actual_data = games.loc[games['year_of_release'] > 2012]
actual_data

## Статистики продаж

In [ ]:
#создадим функцию с выводом статистики, гистрограмм и линейным графиком среднего за год
def analysis(df, col, range_max):
    
    # статистики
    pivot = df.pivot_table(index = 'platform', values = col, aggfunc = ['mean', 'var', 'std','sum'])
    print('Статистики по ', col)
    display(pivot)
    
    # гистограмма
    fig = px.histogram(df, x = col, color = 'platform', marginal = 'box', title = 'Распределение '+col, range_x=[0, range_max])
    fig.show()
    
    # сумма за год
    fig = px.line(df.pivot_table(index = ['platform', 'year_of_release'], values = col, aggfunc = 'sum').reset_index(), x = 'year_of_release', y = col, color = 'platform', title = 'Суммарное по '+col)
    fig.show()

In [ ]:
#применим её к нашим данным за последние 6 лет
analysis(actual_data, 'total_sales', 3)

Из статистики видно, что при средних менее 1 млн копий в год, дисперсия довольна высока (более 2) на некоторых платформах. Что означает частый выпуск хитовых игр, далеко расположенных от среднего.

Из ящиков видно, что значения широко раскинуты у таких платформ, как XBox360, PS3, PS4 и WiiU. А также очень большое количество "выбросов" (хитовые игры не попали в график дабы можно было усмотреть сам ящик).

Суммарные продажи на новых платформах, вышедших в 2013, резко пошли в гору, но к 2016 видно нисходящий тренд, что может означать упадок в игровой индустрии либо неполные данные за год, посмотрев так же на средние значения продаж.

In [ ]:
def sales_pie(data, val):
    (actual_data.pivot_table(index='platform', values=val, aggfunc='sum')
        .sort_values(by=val, ascending=False)
        .head(5)
        .plot.pie(
        autopct="%.2f",
        fontsize=20,
        figsize=(10, 10),
        subplots=True,
        title='Доли продаж на платформах в '+val+' за последние 4 года')
    )
sales_pie(actual_data, 'total_sales')

PS4 уверенно взяла верх по продажам во всём мире - 1/3 от всех продаж. Новое покололение приставок от Microsoft совсем немного (на 2.5%) обошло предущие модели, хотя старт продаж консолей PS и Xbox был в одно время. Возможно у пользователей есть какие-то сомнения или же для Xbox просто меньше игр.

## Выделение акутальных платформ

In [ ]:
#оставим только те платформы, которые существуют на последний год
actual_platforms = games.loc[games['year_of_release'] == 2016]['platform'].unique()
actual_platforms

In [ ]:
actual_data = actual_data.loc[actual_data['platform'].isin(actual_platforms)]
actual_data

## Зависимость продаж от рейтингов

In [ ]:
#выведем матрицу корреляции для продаж и рейтингов
pd.plotting.scatter_matrix(actual_data.loc[:, ['critic_score', 'user_score', 'total_sales']], figsize=(10,10))
actual_data.loc[:, ['critic_score', 'user_score', 'total_sales']].corr()

Как видно между оценками критиков и суммарных продаж зависимость чуть больше, чем пользователей, но всё равно не очень большая. При этом корреляция между рейтингами игроков и критиков более явная.

In [ ]:
#возьмём PS4 как самую популярную по продажам
pd.plotting.scatter_matrix(actual_data.query('platform == "PS4"').loc[:, ['critic_score', 'user_score', 'total_sales']], figsize=(10,10))
actual_data.query('platform == "PS4"').loc[:, ['critic_score', 'user_score', 'total_sales']].corr()

Отдельно для PS4 корелляция продаж и рейтингов критиков выше чем со всеми платформами вместе взятыми.

In [ ]:
pd.plotting.scatter_matrix(actual_data.query('platform == "PC"').loc[:, ['critic_score', 'user_score', 'total_sales']], figsize=(10,10))
actual_data.query('platform == "PC"').loc[:, ['critic_score', 'user_score', 'total_sales']].corr()

А вот для PC зависимость ниже. При этом оценки игроков всё также мало говорят о продажах.

In [ ]:
# статистики
pivot = actual_data.pivot_table(index = 'genre', values = 'total_sales', aggfunc = ['mean', 'var', 'std','count'])
print('Статистики по жанрам')
display(pivot)
    
# гистограмма
fig = px.histogram(actual_data, x = 'total_sales', color = 'genre', marginal = 'box', title = 'Распределение продаж в жанрах', range_x=[0, 4])
fig.show()
    
# сумма за год
fig = px.line(actual_data.pivot_table(index = ['genre', 'year_of_release'], values = 'total_sales', aggfunc = 'sum').reset_index(), x = 'year_of_release', y = 'total_sales', color = 'genre', title = 'Суммарное по продажам в жанрах')
fig.show()

Action игр за последние 4 года выпустили больше всего и своим конвеером игры делают кассу, но шутеры продаются гораздо лучше и средние и медианные продажи в 3 раза выше. Аутсайдерами являются игры класса Adventure, при том что их вышло больше шутеров, средние продажи в более чем 10 раз ниже.

# Портрет регионов

## Северная Америка

In [ ]:
# выведем heatmap суммарных продаж в Северной Америке по жанрам
pivot_heatmap(actual_data, 'genre', 'na_sales')

В Северной Америке самыми популярными жанрами в последние годы являются Action, Shooter и Sports. Тем не менее также видно спад в продажах к 2016 году.

In [ ]:
#статистики
analysis(actual_data, 'na_sales', 2)

Из ящиков видно, что редние продажи по платформам Microsoft (Xbox и Xbox360) выше чем у конкурентов и разброс значений выше. Но опять же средния значения продаж на всех платформах падают к 2016 году.

In [ ]:
#доли продаж по платформам
sales_pie(actual_data, 'na_sales')

Доля продаж у XBox 360, относительно других платформ из топ-5, выше всего - треть. Суммарно по платформам PlayStation и XBox разница не очень велика 40% против 47%.

In [ ]:
#продажи по рейтингам в СА
pivot_heatmap(actual_data, 'rating', 'na_sales')

В Северной Америке более продаются игры с рейтингом М, что означает старше 17 лет.

## Япония

In [ ]:
# heatmap по жанрам
pivot_heatmap(actual_data, 'genre', 'jp_sales')

Как видно, у Японии самые популярные жанры несколько отличаются от Северной Америки. Здесь лидируют RPG и Action. Даже при том, что продажи также к 2016 году снизились по этим жанрам, они всё равно остаются популярными.

In [ ]:
#статистики по Японии
analysis(actual_data, 'jp_sales', 0.5)

Самые высокие средние и суммарные показатели у 3DS. Из десперсии видно, что показатели продаж на ней гораздо шире раскиданы, чем на других платформах. Из линейного графика видно, что WiiU довольно популярна, но, вероятно, спад на рынке снизил средние продажи до показателей других платформ, таких как PS Vita, PS3 и PS4.

In [ ]:
#доли продаж по платформам
sales_pie(actual_data, 'jp_sales')

В Японии безусловным лидером по продажам является платформа компании Nintendo - 3DS. Американские платформы имеют очень малую популярность, а на ПК вообще нет продаж (удивительно).
Топ 5 платформ состоит изключительно из приставок японского производства, где доля 3DS - половина продаж.

In [ ]:
#продажи по рейтингам в Японии
pivot_heatmap(actual_data, 'rating', 'jp_sales')

В Японии большая часть игр продаётся с рейтингом от 14 лет. Популяризация игр начинается от 6 лет.

## Европа

In [ ]:
#heatmap по жанрам
pivot_heatmap(actual_data, 'genre', 'eu_sales')

В Европе картина ровно такая же как и в Америке. Action, Shooters, Sports.

In [ ]:
# статистики
analysis(actual_data, 'eu_sales', 1)

В Европе более успешно продаются игры на PS. Как старого так и нового поколения. На распределении смутил ящик Wii, но оказалось там из-за небольшого количества продаж одна игра в 1.5 млн копий (25% от всех продаж) создала такую картину.

In [ ]:
# топ-5 платформ и доли продаж
sales_pie(actual_data, 'eu_sales')

In [ ]:
#продажи по рейтингам в Европе
pivot_heatmap(actual_data, 'rating', 'other_sales')

Тут картина опять напоминает продажи в СА. От 17 игры продаются в большем количестве.

## Остальной мир

In [ ]:
#heatmap по жанрам
pivot_heatmap(actual_data, 'genre', 'other_sales')

Из карты видно, что остальной мир по предпочтениям близок к Северной Америке. Тут также самыми популярными являются жанры Action, Shooter и Sports. В жанре RPG продажи ниже, но жанр тоже довольно популярен.

In [ ]:
# статистики
analysis(actual_data, 'other_sales', 0.3)

Из ящиков и дисперсии видны широко раскинутые значения на PS3 и PS4. Так же на этих платформах в остальном мире выше средние продажи. Из линейного графика можно наблюдать популярность PS4 на старте продаж платформы и несмотря на спад на игровом рынке в 2016 году средние продажи вдвое больше чем у любого другого конкурента.

In [ ]:
# топ-5 платформ и доли продаж
sales_pie(actual_data, 'other_sales')

Доли продаж на топ-5 платформах в остальном мире похож на североамериканский, если поменять местами платформы Microsoft и Sony местами. В остальном мире более популярны приставки Sony.

In [ ]:
#продажи по рейтингам в остально мире
pivot_heatmap(actual_data, 'rating', 'other_sales')

Во всём остально мире по продажам явно лидируют игры для взрослых (от 17 лет).

# Проверка гипотез

## Средние пользовательские рейтинги платформ Xbox One и PC одинаковые

Выдвенем гипотезу о том, что средние пользовательские рейтинги платформ Xbox One и PC одинаковые. Тогда альтернативной гипотезой будет, что они разные.

In [ ]:
#проведём тест Левена на сравнение дисперсий совокупностей 
stat, p = levene(actual_data.loc[actual_data['platform'] == 'PC']['user_score'].dropna(), 
    actual_data.loc[actual_data['platform'] == 'XOne']['user_score'].dropna())
p

p меньше уровня значимости, следовательно дисперсии не равны и не принадлежат одной генеральной совокупности. Следовательно в тесте Стьюдента необходио указать это.

In [ ]:
alpha = .05 #критический уровень статистической значимости возьмём 5%

results = st.ttest_ind(
    actual_data.loc[actual_data['platform'] == 'PC']['user_score'].dropna(), 
    actual_data.loc[actual_data['platform'] == 'XOne']['user_score'].dropna(),
    equal_var=False)

print('p-значение:', results.pvalue)

if (results.pvalue < alpha):
    print("Отвергаем нулевую гипотезу")
else:
    print("Не получилось отвергнуть нулевую гипотезу") 

p-значение очень высоко. это означает, что даже при том, что среднее по рейтингам отличается, шанс получить такое различие около 62%.

Нулевую гипотезу не отвергаем. Шанс случайно получить различия в средних почти 15%.

## Средние пользовательские рейтинги жанров Action и Sports разные.

Нулевая гипотеза звучит как "Средние пользовательские рейтинги жанров Action и Sports одинаковые"
Нас интересует альтернативная гипотеза, что средние значения жанров разные. 

In [ ]:
#проведём тест Левена на сравнение дисперсий совокупностей 
stat, p = levene(actual_data.loc[actual_data['genre'] == 'Action']['user_score'].dropna(), 
    actual_data.loc[actual_data['genre'] == 'Sports']['user_score'].dropna())
p

p меньше уровня значимости, следовательно дисперсии не равны и не принадлежат одной генеральной совокупности.

In [ ]:
results = st.ttest_ind(
    actual_data.loc[actual_data['genre'] == 'Action']['user_score'].dropna(), 
    actual_data.loc[actual_data['genre'] == 'Sports']['user_score'].dropna(),
    equal_var=False)

print('p-значение:', results.pvalue)

if (results.pvalue < alpha):
    print("Отвергаем нулевую гипотезу")
else:
    print("Не получилось отвергнуть нулевую гипотезу") 

Полученное значение p-value говорит о том, что с такой маленькой вероятностью ( 2.4е-20 %) такое или большее различие можно получить случайно, т.е. вероятно они неслучайны. Это явно слишком маленькая вероятность, поэтому отвергаем нулевую гипотезу на уровне значимости 0,05 и принимаем альтернативную гипотезу: средние пользовательские рейтинги жанров Action и Sports разные.

# Общий вывод

Как мы выяснили в Северной Америке предпочтения по жанрам и платформам схожы с предпочтениями во всём мире (не включая Японию). В Америке чуть более популярны приставки Microsoft, в остальном мире Sony, в Японии платформы собственного производства (лидером является Nintendo 3DS). Самый популярный жанр во всём мире является Action. В Японии это также RPG. В остальном мире, включая СА это Shooter и Sports. Продажи по возрастному рейтингу гораздо выше для игр от 17 лет. Продажи не зависят от рейтингов игроков и критиков. Североамериканский рынок самый крупный по продажам за последние 4 года и составляет почти 2/3 от продаж во всё мире, а при том что рынок в остальном мире схож по вкусам с СА, то при формировании стратегии рекламы стоит ориентироваться на их вкусы ии предпочтения. Для Японии же нужен индивидуальный подход.

Мы выдвинули две гипотезы:
1. Что средние пользовательские рейтинги платформ Xbox One и PC одинаковые. Их отличия несущественны и могут быть на 62% случайны.
2. Средние пользовательские рейтинги жанров Action и Sports разные на уровне значимости 0.05%. Эту гипотезу мы подтвердили.